<a href="https://colab.research.google.com/github/stevencao2005/DeepLearningModels_On_EEG_Data/blob/main/Deep_Learning_Networks_on_EEG_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the modules and python files

In [ ]:
#IMPORT ALL NEEDED MODULES
import os
import numpy as np
import time
import sys
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import tensorboard
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler


In [ ]:
#MOUNT GOOGLE DRIVE TO IMPORT FILES IN OUR GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#ALLOWING THE IMPORT OF PYTHON FILES
!cp /content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/utils.py .
sys.argv.extend('/content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/')
sys.argv.extend('/content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/classifiers')

In [ ]:
!cp /content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/classifiers/inception.py .
!cp /content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/classifiers/resnet.py .
!cp /content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/classifiers/EEGModels.py .

In [ ]:
#IMPORTING PYTHON FILES
import utils
from utils import generate_results_csv
from utils import create_directory
from utils import read_dataset
from utils import transform_mts_to_ucr_format
from utils import visualize_filter
from utils import viz_for_survey_paper
from utils import calculate_metrics
from utils import case_by_case_analysis
from utils import viz_cam

## Functions used

In [ ]:
def one_hot_encoder(y_train, y_test):
    enc     = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test  = enc.transform(y_test.reshape(-1, 1)).toarray()
    return y_train, y_test
def standardizing_the_dataset(X_train):
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    return X_train
#FOR FITTING THE MODEL ONTO THE DATASET
def fit_classifier():
    #GETTING THE TRAIN/TEST DATASETS
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test  = datasets_dict[dataset_name][2]
    y_test  = datasets_dict[dataset_name][3]

    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

    #TRANSFROM THE LABELS FROM INTEGERS TO ONE HOT VECTORS
    y_train, y_test = one_hot_encoder(y_train, y_test)

    #standardizing the train/test datasets for each channel
    x_train_preprocessed = np.zeros((x_train.shape[0], x_train.shape[1], x_train.shape[2]))
    x_test_preprocessed  = np.zeros((x_test.shape[0], x_test.shape[1], x_test.shape[2]))
    channels             = [i for i in range(x_test.shape[1])]
    for channel in channels:
        x_train_channelData       = x_train[:,channel,:]
        x_test_channelData        = x_test[:,channel,:]
        x_train_channelData       = np.expand_dims(standardizing_the_dataset(x_train_channelData), axis=1)
        x_test_channelData        = np.expand_dims(standardizing_the_dataset(x_test_channelData), axis=1)
        x_train_preprocessed[:,channel:channel+1, :] = x_train_channelData
        x_test_preprocessed[:,channel:channel+1,:]   = x_test_channelData
    x_train = x_train_preprocessed
    x_test  = x_test_preprocessed 
     # save orignal y because later we will use binary
    y_true  = np.argmax(y_test, axis=1)

    #INITIALIZING THE CLASSIFIER TO BE TRAINED ON
    input_shape = x_train.shape[1:]
    classifier = create_classifier(classifier_name, input_shape, nb_classes, output_directory)

    #FITTING THE CLASSIFIER USING THE PREPROCESSED DATA
    classifier.fit(x_train, y_train, x_test, y_test, y_true)


#FOR AFTER FITTING THE MODEL, AND WANT TO LOAD THE MODEL AND ANALYZE ITS PERFORMANCE
def fitted_classifier():
    #GETTING THE TRAIN/TEST DATASETS
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]

    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

    #TRANSFROM THE LABELS FROM INTEGERS TO ONE HOT VECTORS
    y_train, y_test = one_hot_encoder(y_train, y_test)

    #standardizing the train/test datasets for each channel
    x_train_preprocessed = np.zeros((x_train.shape[0], x_train.shape[1], x_train.shape[2]))
    x_test_preprocessed  = np.zeros((x_test.shape[0], x_test.shape[1], x_test.shape[2]))
    channels             = [i for i in range(x_test.shape[1])]
    for channel in channels:
        x_train_channelData       = x_train[:,channel,:]
        x_test_channelData        = x_test[:,channel,:]
        x_train_channelData       = np.expand_dims(standardizing_the_dataset(x_train_channelData), axis=1)
        x_test_channelData        = np.expand_dims(standardizing_the_dataset(x_test_channelData), axis=1)
        x_train_preprocessed[:,channel:channel+1, :] = x_train_channelData
        x_test_preprocessed[:,channel:channel+1,:]   = x_test_channelData
    x_train = x_train_preprocessed
    x_test  = x_test_preprocessed 
    # save orignal y because later we will use binary
    y_true = np.argmax(y_test, axis=1)

    #INITIALIZING THE CLASSIFIER TO BE TRAINED ON
    input_shape = x_train.shape[1:]
    classifier = create_classifier(classifier_name, input_shape, nb_classes, output_directory)

    #LOADING THE MODEL AND EVALUATE IT USING THE TEST DATASET
    y_pred, df_metrics = predict(x_test, y_true, x_train, y_train, y_test, output_directory, return_df_metrics=True)

    return x_test, y_true, x_train, y_train, y_test, y_pred, df_metrics

def predict(x_test, y_true, x_train, y_train, y_test, output_directory, return_df_metrics=True):

    #LOADING THE MODEL
    start_time = time.time()
    model_path = output_directory + 'best_model.h5'
    model      = keras.models.load_model(model_path)

    #PREDICTING THE SAMPLES FROM THE TEST DATASET
    time1  = time.time()
    y_pred = model.predict(x_test, batch_size=64)
    time2  = time.time()
    print("time to go through the test set:", time2-time1)
    if return_df_metrics:
        y_pred     = np.argmax(y_pred, axis=1)
        df_metrics = calculate_metrics(y_true, y_pred, 0.0)
        return y_pred, df_metrics


def create_classifier(classifier_name, input_shape, nb_classes, output_directory, verbose=True):
  # changed from "from classifier import fcn"    to    "import fcn"
    #GETTING THE CLASSIFIER THAT WE WANT TO BE TRAINED ON THE DATA
    if classifier_name == 'resnet':
        import resnet
        return resnet.Classifier_RESNET(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'inception':
        import inception
        return inception.Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'eegnet':
        import EEGModels
        return EEGModels.Classifier_EEGNET(output_directory, input_shape, nb_classes, verbose, nb_epochs=1000)

def read_dataset(root_dir, dataset_name):
    #LOAD THE DATASET AND SPLIT IT INTO TRAIN/TEST DATASETS
    datasets_dict = {}
    cur_root_dir  = root_dir
    file_name     = cur_root_dir + '/' + dataset_name + '/'
    x_train       = np.load(file_name + 'X_train.npy')
    y_train       = np.load(file_name + 'y_train.npy')
    x_test        = np.load(file_name + 'X_test.npy')
    y_test        = np.load(file_name + 'y_test.npy')

    datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                   y_test.copy())
    return datasets_dict

## Main Code

In [ ]:
#--STARTING CODE ---- 
root_dir = '/content/gdrive/MyDrive/DeepLearningModels_On_EEG_Data/'

#DIRECTIONS ON THE DATASET AND CLASSIFIER TO BE USED
pathway = []
pathway.extend(['BED','eegnet'])  #it can be inception, resnet, eegnet

In [ ]:
#INITIALIZING VARIABLES FOR CREATING A DIRECTORY TO PUT THE RESULTS OF THE MODEL IN
dataset_name     = pathway[0]
classifier_name  = pathway[1]
output_directory = root_dir + '/results/' + classifier_name + '/' +  dataset_name + '/'
test_dir_df_metrics = output_directory + 'df_metrics.csv'

#IF THE MODEL HAS ALREADY BEEN TRAINED ON, THEN WE CAN LOAD IT AND ANALYSIS ITS PERFORMANCE. IF NOT TRAINED YET, THEN WE CAN TRAIN THE MODEL.
if os.path.exists(test_dir_df_metrics):
    print('Already done')
    loadModelAnswer = input("Would you like to load the best model:  (y/n)")
    if loadModelAnswer == 'y':
        datasets_dict = read_dataset(root_dir, dataset_name)
        x_test, y_true, x_train, y_train, y_test, y_pred, df_metrics = fitted_classifier()
        predictions = case_by_case_analysis(y_true, y_pred) #predictions[0] to see the correct and incorrect predictions.
        i=1
else:
    #CREATING THE OUTPUT DIRECTORY WHERE ALL THE RESULTS FROM THE MODEL BEING TRAINED WILL BE LOCATED
    create_directory(output_directory)

    # READING THE DATASET AND SPLITTING IT INTO TRAIN/TEST DATASETS
    datasets_dict = read_dataset(root_dir, dataset_name)

    #FITTING THE CLASSIFIER ONTO THE DATASET AND GETTING THE PERFORMANCE OF THE MODEL
    fit_classifier()

    #CREATING A DIRECTORY TO SAY THAT WE FINISHED TRAINING AND EVALUATING THE MODEL
    print('DONE')
    create_directory(output_directory + '/DONE')

NameError: ignored

In [ ]:
dataset_name     = pathway[0]
classifier_name  = pathway[1]
output_directory = root_dir + '/results/' + classifier_name + '/' +  dataset_name + '/'
test_dir_df_metrics = output_directory + 'df_metrics.csv'
datasets_dict = read_dataset(root_dir, dataset_name)

In [ ]:
aa = datasets_dict[dataset_name][2]
print(aa.shape)